In [ ]:
import torch
import csv
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm import trange
import matplotlib.style as mplstyle
import math
import random

In [ ]:
device=''
if torch.cuda.is_available():
  device = torch.cuda.current_device()
else:
  device='cpu'

In [ ]:
path_train="./drive/MyDrive/ml/lab5/training_3.text"
dim=2

In [ ]:
class MyDataset(Dataset):
  def __init__(self,path):
    self.input_data=[]
    self.output_data=[]
    with open(path,"r") as f:
      for line in f:
        line=line.replace('\n','')
        data=line.split(' ')
        linshi=[]
        for i in range(len(data)-1):
          linshi.append(float(data[i]))
        self.input_data.append(linshi)
        self.output_data.append(float(data[-1]))
  def __len__(self):
    return len(self.input_data)
  def __getitem__(self,index):
    x=self.input_data[index]
    y=self.output_data[index]
    ans_x=torch.Tensor(x)
    ans_y=torch.Tensor(1)
    ans_y[0]=y
    return ans_x,ans_y

In [ ]:
dataset=MyDataset(path_train)
batch_size=len(dataset)
dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=False)

In [ ]:
print(batch_size)

In [ ]:
for i,data in enumerate(tqdm(dataloader)):
  for j in range(batch_size):
    if data[1][j][0].item()==1:
      plt.scatter(data[0][j][0].item(),data[0][j][1].item(),c='r')
    if data[1][j][0].item()==-1:
      plt.scatter(data[0][j][0].item(),data[0][j][1].item(),c='g')
plt.show()

In [ ]:
gamma=100

In [ ]:
front_one=torch.zeros((batch_size*batch_size,1))
front_two=torch.zeros((batch_size*batch_size,1))
front=torch.zeros((batch_size*batch_size,1))
for a,b in enumerate(dataloader):
  y=b[1]
  y=y.reshape(batch_size,1)
  y_t=torch.transpose(y,1,0)
  yy_t=y@y_t


  x=b[0]
  x_t=torch.transpose(x,1,0)
  xx_t=-2*x@x_t



  x_first=b[0].reshape(batch_size,1,dim)
  x_first_2=torch.bmm(x_first,torch.transpose(x_first,2,1))
  x_first_2=x_first_2.reshape(1,batch_size)



  x_first_2=x_first_2.repeat(batch_size,1)


  x_second_2=torch.transpose(x_first_2,0,1)


  sum_ans=x_first_2+xx_t+x_second_2



  sum_ans=-gamma*sum_ans
  end_ans=np.exp(sum_ans)


  front=yy_t*end_ans

In [ ]:
front=front.detach()
front=front.to(device)
para=torch.zeros((len(dataset),1),requires_grad=True)
para=para.to(device)
para=para.detach()
para.requires_grad=True

In [ ]:
def loss(x,y):
  sum_a=-torch.sum(para,dim=0)
  end_one=para@torch.transpose(para,1,0)
  end_two=0.5*front*end_one
  end_ans=end_two.reshape(batch_size*batch_size,1)
  ls2=sum_a+torch.sum(end_ans,dim=0)
  return ls2

In [ ]:
learn_rating=1e-3
epoch=10000

In [ ]:
for d,w in enumerate(dataloader):
  j=w
for n in trange(epoch):
  j[0]=j[0].to(device)
  j[1]=j[1].to(device)
  lo=loss(j[0],j[1])
  lo.backward()
  torch.nn.utils.clip_grad_norm_(para.grad,max_norm=2,norm_type=2)
  if n%2000==0:
    print(lo)
  with torch.no_grad():
    o=0
    o_num=0
    p=random.randint(0,batch_size-1)
    for i in range(batch_size):
      if torch.abs(para.grad[i])>o_num:
        o=i
        o_num=torch.abs(para.grad[i])
    while o==p:
      p=random.randint(0,batch_size-1)
    li=para[o].item()
    li2=para[p].item()
    now_sum=para[o]*j[1][o]+para[p]*j[1][p]
    para[o]=para[o]-para.grad[o]*learn_rating
    para[p]=(now_sum-para[o]*j[1][o])/j[1][p]
    if para[o]<0 or para[p]<0:
      para[o]=li
      para[p]=li2
  para.grad.zero_()

In [ ]:
def get_ans(data,x,y):
  y=y.reshape(batch_size,1)
  data=data.reshape(batch_size,dim)
  x=x.reshape(1,dim)
  x_input=x.repeat(batch_size,1)
  cha=data-x_input
  norm2=cha**2
  norm2=torch.sum(norm2,dim=1)
  norm2=norm2.reshape(batch_size,1)
  norm2=torch.exp(-gamma*norm2)
  ans_mid=para*y*norm2
  ans_mid=ans_mid.reshape(batch_size,1)
  return torch.sum(ans_mid,dim=0)

In [ ]:
for i,data in enumerate(dataloader):
  data[0]=data[0].to(device)
  data[1]=data[1].to(device)
  min_positive=1e18
  max_negative=-1e18
  for j in range(batch_size):
    if data[1][j][0].item()==1:
      linshi=data[0][j].reshape(1,2)
      now=get_ans(data[0],linshi,data[1])
      if now[0].item()<min_positive:
        min_positive=now[0].item()
    if data[1][j][0].item()==-1:
      linshi=data[0][j].reshape(1,2)
      now=get_ans(data[0],linshi,data[1])
      if now[0].item()>max_negative:
        max_negative=now[0].item()
  bias=(min_positive+max_negative)/-2
print(bias)

In [ ]:
for d,data in enumerate(dataloader):
  for j in range(batch_size):
    if data[1][j][0].item()==1:
      plt.scatter(data[0][j][0].item(),data[0][j][1].item(),c='r')
    if data[1][j][0].item()==-1:
      plt.scatter(data[0][j][0].item(),data[0][j][1].item(),c='g')
  data=data
data[0]=data[0].to(device)
data[1]=data[1].to(device)
for i in tqdm(np.arange(-0.5,0.2,0.001)):
  for j in np.arange(-0.6,0.6,0.001):
    now=torch.Tensor([i,j])
    now=now.to(device)
    ans=get_ans(data[0],now,data[1])+bias
    if ans[0].item()>=-0.0000001 and ans[0].item()<=0.0000001:
      plt.scatter(i,j,s=5,c='b')
plt.show()